In [14]:
from elasticsearch import Elasticsearch 
from elasticsearch.helpers import streaming_bulk


es = Elasticsearch()

In [62]:
import ipywidgets as ipw
from typing import Dict


def create_image_carousel(image_data: Dict[str, str]) -> ipw.HBox:
    carousel_child = []
    for img in image_data:
        img_box = ipw.Image.from_url(img['url'])
        desc = img['description']
        text_box = f'<p style="font-family:sans-serif">{desc}</p>'
        desc_box = ipw.HTML(value=text_box)
        info_box = ipw.VBox([img_box, desc_box])
        carousel_child.append(info_box)

    return ipw.HBox(carousel_child)


def perform_painting_search(index: str, query: str):
    query = {
        "size": 10,
        "query": {
            "match": {
                "description": query
            }
        },
        "aggs": {
            "authors": {
                "terms": {
                    "field": "author"
                }
            },
            "genres": {
                "terms": {
                    "field": "genre"
                }
            },
            "Total authors": {
                "cardinality": {
                    "field": "author"
                }
            },
            "Styles": {
                "cardinality": {
                    "field": "style"
                }
            }
        }

    }
    res = es.search(index=index, body=query)
    print(res['aggregations'])
    img_data = [x['_source'] for x in res['hits']['hits']]
    return img_data


img_data = perform_painting_search(index="paintings", query="vase")
create_image_carousel(img_data)


/opt/homebrew/Caskroom/miniforge/base/envs/.art/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'genres': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': 'mythological painting, utensil', 'doc_count': 8}, {'key': 'figurative', 'doc_count': 1}, {'key': 'genre painting', 'doc_count': 1}, {'key': 'genre painting, utensil', 'doc_count': 1}, {'key': 'sculpture', 'doc_count': 1}]}, 'Total authors': {'value': 3}, 'Styles': {'value': 6}, 'authors': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': '\uffff', 'doc_count': 13}, {'key': 'Bunny Rupert ', 'doc_count': 1}, {'key': 'Chicote CFC', 'doc_count': 1}]}}


In [50]:
es.indices.get_mapping( 'paintings')

/opt/homebrew/Caskroom/miniforge/base/envs/.art/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'paintings': {'mappings': {'properties': {'author': {'type': 'keyword'},
    'description': {'type': 'text'},
    'genre': {'type': 'keyword'},
    'material': {'type': 'keyword'},
    'style': {'type': 'keyword'},
    'tags': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'title': {'type': 'text'},
    'url': {'type': 'text'}}}}}